# Intro to Gurobi — LP Exercises (Workshop)

**Please read before class:**

- Review **Q1** (worked example) and try to run it.
- Start **Q2–Q3**. Bring your questions to the workshop.

During the 3‑hour session, you'll continue with **Q2–Q6** in groups. We’ll circulate to help.

---
### LP modeling mindset
**Decisions → Objective → Constraints**

Keep asking:
- *What are the decision variables?*
- *What is the objective?*
- *What are the constraints?*


In [6]:
# === SETUP (Colab/Local) ===
try:
    import gurobipy as gp
    from gurobipy import GRB
except Exception:
    !pip install gurobipy
    import gurobipy as gp
    from gurobipy import GRB
print('Gurobi version:', gp.gurobi.version())
# Optional: fetch academic key once on a machine
# !grbgetkey YOUR_KEY


Gurobi version: (13, 0, 0)


## Production Planning Exercise from the lecture   



### Poor style: 

In [4]:
from gurobipy import*
model=Model()
wrenches=model.addVar(vtype=GRB.CONTINUOUS)
plyers=model.addVar(vtype=GRB.CONTINUOUS)
model.setObjective(0.13*wrenches+0.1*plyers,GRB.MAXIMIZE)
model.addConstr(1*wrenches+1*plyers<=21000)
model.addConstr(0.3*wrenches+0.5*plyers<=9000)
model.addConstr(1.5*wrenches+1*plyers<=27000)
wrenches.UB=15000
plyers.UB=16000
wrenches.LB=0
plyers.LB=0
model.optimize()

print("wrenches to produce",wrenches.X)
print("plyers to produce",plyers.X)
print("Optimal Revenue",model.ObjVal)

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[arm] - Darwin 24.6.0 24G84)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 3 rows, 2 columns and 6 nonzeros (Max)
Model fingerprint: 0xed3ea001
Model has 2 linear objective coefficients
Coefficient statistics:
  Matrix range     [3e-01, 2e+00]
  Objective range  [1e-01, 1e-01]
  Bounds range     [2e+04, 2e+04]
  RHS range        [9e+03, 3e+04]
Presolve time: 0.00s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.7300000e+03   2.062500e+03   0.000000e+00      0s
       2    2.4600000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.460000000e+03
wrenches to produce 12000.0
plyers to produce 9000.000000000002
Optimal Revenue 2460.0000000000005


### Better style to solve production problem

In [9]:
#setup 
from gurobipy import *

tools = ["wrenches", "pliers"]
resources = ["steel", "molding", "assembling"]

r = {
    ("steel", "wrenches"): 1.5,
    ("steel", "pliers"): 1,
    ("molding", "wrenches"): 1,
    ("molding", "pliers"): 1,
    ("assembling", "wrenches"): 0.3,
    ("assembling", "pliers"): 0.5
}

availability = {
    "steel": 27000,
    "molding": 21000,
    "assembling": 9000
}

demand = {
    "wrenches": 15000,
    "pliers": 16000
}

price = {
    "wrenches": 0.13,
    "pliers": 0.1
}


#Implementation
model = Model()
x=model.addVars(tools, obj=price, ub=demand)
model.ModelSense = GRB.MAXIMIZE
resourceCons=model.addConstrs(quicksum(r[i,j]*x[j] for j in tools)<=availability[i] for i in resources)
model.optimize()



print("\nOptimal production plan:")
for j in tools:
    print(f"  Produce {x[j].X} units of {j}")

print(f"\nTotal profit: {model.ObjVal}")

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[arm] - Darwin 24.6.0 24G84)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 3 rows, 2 columns and 6 nonzeros (Max)
Model fingerprint: 0x01a17661
Model has 2 linear objective coefficients
Coefficient statistics:
  Matrix range     [3e-01, 2e+00]
  Objective range  [1e-01, 1e-01]
  Bounds range     [2e+04, 2e+04]
  RHS range        [9e+03, 3e+04]
Presolve time: 0.00s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.7000000e+03   3.250000e+03   0.000000e+00      0s
       3    2.4600000e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.460000000e+03

Optimal production plan:
  Produce 12000.0 units of wrenches
  Produce 9000.000000000002 units of pliers

Total profit: 2460.0000000000005


## Q1 — Production Planning (Demonstration)
**Question 1: Galaxy Industries** would like to determine production levels for four of its toy water guns that will maximize the total profit. Galaxy wants to produce at least 100 units and at most 1000 units of each toy water gun. The following table summarizes the profits and the resources requirements. The last row provides the resources available per week.

| Product | Profit | Plastic (lbs.) | Production time (min) |
|---|---:|---:|---:|
| Space Ray | €16 | 2 | 3 |
| Zapper    | €15 | 1 | 4 |
| Big Squire| €20 | 3 | 5 |
| Soaker    | €22 | 4 | 6 |
| **Available** |  | **3000** | **6000** |

1. Formulate an LP model for this problem.

2. Solve the problem using Gurobi.

3. What is the optimal solution?


## Model

- We must decide how much to produce for each toy. Use $x_1,x_2,x_3,x_4$ for Space Ray, Zapper, Big Squire, Soaker respectively. The model is

$$max \quad 16 x_1 +15 x_2 + 20 x_3 + 22 x_4$$
s.t.
$$ \quad 2 x_1 +x_2 +3x_3 +4x_4 \leq 3000$$
$$ \quad 3 x_1 +4 x_2 +5x_3 +6x_4 \leq 6000$$
$$100\leq x_i \leq 1000, \quad \forall i \in \{1,2,3,4\} $$

### Q1 — Way 1: Direct Gurobi model (Decisions → Objective → Constraints)

In [3]:
model=gp.Model()
x1=model.addVar(lb=100,ub=1000)
x2=model.addVar(lb=100,ub=1000)
x3=model.addVar(lb=100,ub=1000)
x4=model.addVar()
x4.lb=100
x4.ub=1000
model.setObjective(16*x1+15*x2+20*x3+22*x4,gp.GRB.MAXIMIZE)
model.addConstr(2*x1+x2+3*x3+4*x4<=3000)
model.addConstr(3*x1+4*x2+5*x3+6*x4<=6000)
model.optimize()
print("problem status is", model.status)
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("x1",x1.X)
print("x1",x2.X)
print("x1",x3.X)
print("x1",x4.X)
print("optimal value",model.objval)

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G84)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2 rows, 4 columns and 8 nonzeros
Model fingerprint: 0xe7d9c8b2
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+02, 1e+03]
  RHS range        [3e+03, 6e+03]
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7300000e+04   5.125000e+02   0.000000e+00      0s
       2    2.6660000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.666000000e+04
problem status is 2
x1 860.0
x1 580.0
x1 100.0
x1 100.0
optimal value 26660.0


### Q1 — Way 2 (better): Using Dictionaries

In [4]:
from gurobipy import*
toys=["Space Ray", "Zapper", "Big Squire", "Soaker"]
resources=["plastic", "production time"]
price={"Space Ray":16, "Zapper":15, "Big Squire":20, "Soaker":22}
availability={"plastic":3000, "production time":6000}
r={("plastic","Space Ray"):2, ("plastic","Zapper"):1, ("plastic","Big Squire"):3, ("plastic","Soaker"):4,
   ("production time","Space Ray"):3, ("production time","Zapper"):4, ("production time","Big Squire"):5, ("production time","Soaker"):6}

model=Model()
x=model.addVars(toys, lb=100, ub=1000, obj=price,name="toys")
model.ModelSense=GRB.MAXIMIZE
resourceCons=model.addConstrs(quicksum(r[i,j]*x[j] for j in toys)<=availability[i] for i in resources)
model.optimize()


if not model.status ==GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")


Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[arm] - Darwin 24.6.0 24G84)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2 rows, 4 columns and 8 nonzeros (Max)
Model fingerprint: 0xe7d9c8b2
Model has 4 linear objective coefficients
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+02, 1e+03]
  RHS range        [3e+03, 6e+03]
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7300000e+04   5.125000e+02   0.000000e+00      0s
       2    2.6660000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.666000000e+04
optimal value 26660.0

    Variable            X 
-------------------------
toys[Space Ray]          860 
toys[Zapper]          580 
toys[Big Squire]          100 
toys[Soak

### Q1 — Way 3: Using for loops (some times more flexible)

In [ ]:
#Data
products=["Ray","Zapper","Squire","Soaker"]
resources=["plastic","time"]
profit={"Ray":16
        ,"Zapper":15
        ,"Squire":20,
        "Soaker":22}
resAvail={"plastic":3000,"time":6000}
resUse={("Ray","plastic"):2,("Ray","time"):3,
    ("Zapper","plastic"):1,("Zapper","time"):4,
    ("Squire","plastic"):3,("Squire","time"):5,
    ("Soaker","plastic"):4,("Soaker","time"):6
    }

#Model
model=gp.Model()
pvars={}
for p in products:
    pvars[p]=model.addVar(lb=100,ub=1000,obj=profit[p],name=p)
model.ModelSense=gp.GRB.MAXIMIZE
for r in resources:
    lhs=0
    for p in products:
        lhs+=resUse[p,r]*pvars[p]
    model.addConstr(lhs<=resAvail[r])
model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 2 rows, 4 columns and 8 nonzeros
Model fingerprint: 0xe7d9c8b2
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+02, 1e+03]
  RHS range        [3e+03, 6e+03]
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7300000e+04   5.125000e+02   0.000000e+00      0s
       2    2.6660000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.666000000e+04
optimal value 26660.0

    Variable            X 
-------------------------
         Ray          860 
      Zapper          580 
      Squire          100 
      Soaker          100 


### Q1 — Way 4: Using arrays. A matter of taste which one you prefer. Good to know both as sometimes one may be more convenient than the other.

In [ ]:
import numpy as np
profit = np.array([16,15, 20, 22])
res_use=np.array([[2,1,3,4],[3,4,5,6]])
res_avail = np.array([3000, 6000])
n = len(profit)                                    # number of variables
m = len(res_avail)                                 # number of constraints
assert res_use.shape == (m, n)
model=gp.Model("Q1")
pvars=model.addVars(n,name="products",lb=100,ub=1000)
model.setObjective(gp.quicksum(profit[i]*pvars[i] for i in range(n)),gp.GRB.MAXIMIZE)
cons=model.addConstrs(gp.quicksum(res_use[i,j]*pvars[j] for j in range(n))<=res_avail[i]
                     for i in range(m))
model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 2 rows, 4 columns and 8 nonzeros
Model fingerprint: 0xe7d9c8b2
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+02, 1e+03]
  RHS range        [3e+03, 6e+03]
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7300000e+04   5.125000e+02   0.000000e+00      0s
       2    2.6660000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.666000000e+04
optimal value 26660.0

    Variable            X 
-------------------------
 products[0]          860 
 products[1]          580 
 products[2]          100 
 products[3]          100 


## Q2 — Outsourcing with Capacity (High Guidance)
**Question 2:** Your company makes a variety of products. A large order for three products has just been received from a customer who also requested a very short due date. A quick calculation revealed that the limiting resource is the labor time; only **200 labor hours** are available, and that is not enough to make all the requested units of products. Some units must be outsourced to subcontractors. Two subcontractors are available. Subcontractor **A** has low prices, but can only provide a maximum of **60 units** of all products in total. Sub-contractor **B** is more expensive, but can provide any number of units of any product. Your company wants to decide how many units of each product to make and how many units to buy from each subcontractor in order to **minimize the total cost**. The table below summarizes all necessary information.

|  | Product 1 | Product 2 | Product 3 |
|---|---:|---:|---:|
| Cost of making a unit | €6 | €13 | €20 |
| Cost of buying a unit from A | €12 | €15 | €21 |
| Cost of buying a unit from B | €11 | €16 | €23 |
| Labor hours / unit | 1 | 2 | 3 |
| Demand (units) | 100 | 80 | 70 |



### Model (given)

**Start with plain English.**  
Always think: *What are my decisions? What is my objective? What are my constraints?*

**Decisions (plain English):**  
For **each product** \(i \in \{1,2,3\}\), decide
- how many units to **make in-house**,
- how many units to **buy from Subcontractor A**,
- how many units to **buy from Subcontractor B**.

**Variables (mathematical):**
- $M_i \geq 0$: units of product \(i\) **made** in-house  
- $A_i \geq 0$: units of product \(i\) **bought from A**  
- $B_i \geq 0$: units of product \(i\) **bought from B**

**Parameters (from the table/problem statement):**
- Unit costs: $c_i^{m}$ (make), $c_i^{A}$ (buy from A), $c_i^{B}$ (buy from B)
- Labor hours per made unit: $\ell_i$ (here $\ell_1=1,\ell_2=2, \ell_3=3)$
- Available labor: \(200\) hours
- Subcontractor A total capacity: \(60\) units
- Demand for each product: \(d_i\) (here $d_1=100,\ d_2=80,\ d_3=70$)

**Objective (what we optimize):**  
Minimize the **total cost** of meeting demand:
$$
\min \sum_{i=1}^3 \big(c_i^{m} M_i + c_i^{A} A_i + c_i^{B} B_i\big).
$$

**Constraints (what must hold):**
1) **Labor limit (in-house production consumes labor):**
$$
\ell_1 M_1 + \ell_2 M_2 + \ell_3 M_3 \leq 200
\quad\text{(here: }1 M_1 + 2 M_2 + 3 M_3 \leq 200\text{)}.
$$

2) **Subcontractor A capacity (total bought from A across all products):**
$$
A_1 + A_2 + A_3 \leq 60.
$$

3) **Demand balance (for each product, supply must meet demand):**
$$
M_i + A_i + B_i = d_i, \quad \forall i \in \{1,2,3\}.
$$

4) **Nonnegativity:**
$$
M_i, A_i, B_i \ge 0, \quad \forall i.
$$

---

### Overall compact formulation


\begin{aligned}
\min_{M_i, A_i, B_i}\quad
& \sum_{i=1}^3 \big(c_i^{m} M_i + c_i^{A} A_i + c_i^{B} B_i\big) \\
\text{s.t.}\quad
& \sum_{i=1}^3 \ell_i M_i \le 200 \quad  \\
& \sum_{i=1}^3 A_i \le 60 \\
& M_i + A_i + B_i = d_i, \quad i=1,2,3 \\
& M_i, A_i, B_i \ge 0, \quad i=1,2,3.
\end{aligned}


In [22]:
# 🧩 Your Turn (Q2)
# Complete the implementation of the given model in gurobipy and solve.

products=[1,2,3]
makeCosts={1:6,2:13,3:20}
buyACosts={1:12,2:15,3:21}
buyBCosts={1:11,2:16,3:23}
demands={1:100,2:80,3:70}
laborUse={1:1,2:2,3:3}
AproductionLimit=60
laborAvailability=200

model=gp.Model()
M=model.addVars(products,obj=makeCosts, name="make")
A=model.addVars(products,obj=buyACosts, name="Contractor A")
B=model.addVars(products,obj=buyBCosts, name="Contractor B")
model.addConstrs(M[p]+A[p]+B[p]==demands[p] for p in products)
model.addConstr(gp.quicksum(laborUse[p]*M[p] for p in products)<= laborAvailability)
model.addConstr(gp.quicksum(A[p] for p in products)<=AproductionLimit)
model.optimize()

model.printAttr("X")





Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[arm] - Darwin 24.6.0 24G84)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 5 rows, 9 columns and 15 nonzeros (Min)
Model fingerprint: 0xba04db3c
Model has 9 linear objective coefficients
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [6e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+02]
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolved: 4 rows, 7 columns, 11 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0400000e+03   1.769142e+01   0.000000e+00      0s
       4    3.2200000e+03   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.220000000e+03

    Variable            X 
-------------------------
     make[1]          100 
     make[2]           50 
Contractor A[3]          

## Q3 — Portfolio Allocation (Medium Guidance)
**Question 3:** A trust officer at the Maltese National Bank needs to determine how to invest **€100,000** in the following collection of bonds to **maximize the annual return**.

| Bond | Annual return | Maturity | Risk | Tax-Free |
|---|---:|---|---|---|
| A | 9.5% | Long | High | Yes |
| B | 8%   | Short| Low  | Yes |
| C | 9%   | Long | Low  | No  |
| D | 9%   | Long | High | Yes |
| E | 9%   | Short| High | No  |

The officer wants to invest **at least 50%** of the money in short-term issues and **no more than 45%** in high risk issues. At least **30%** of the funds should go into tax-free investments and at least **40%** of the total annual return should be tax-free.



**Start of model (partial):**

- **Decisions:** invest amounts in bonds A–E (variables A,B,C,D,E ≥ 0)
- **Objective:** maximize total annual return (use decimal returns, e.g., 0.095 for 9.5%)
- **Constraints (to complete):**
  - Budget: A + B + C + D + E = 100,000
  - Short-term (B and E): ≥ 50% of total budget
  - High risk (A, D, E): ≤ 45% of total budget
  - Tax-free funds (A, B, D): ≥ 30% of total budget
  - **Tax-free return requirement:** return from (A,B,D) ≥ 40% of total return


In [ ]:
#Finish the Model
Bonds=["A","B","C","D","E"]
n=len(Bonds)
returns=[.095,.08,.09,.09,.09]
longMaturity=[True,False,True,True,False]
highRisk=[1,0,0,1,1]
taxFree=[1,1,0,1,0]

totalInvestment=100000
shortTermMinRatio=.5
lowRiskRatio=.55
taxFreeMinRatio=.3
taxFreeRetMinRatio=.4

model=gp.Model()
x=model.addVars(n,name=Bonds,obj=returns)
model.ModelSense=gp.GRB.MAXIMIZE
model.addConstr(gp.quicksum(x[i] for i in range(n))==totalInvestment)
model.addConstr(gp.quicksum(x[i] for i in range(n) if not longMaturity[i])
                >=totalInvestment*shortTermMinRatio)










## Q4 — Transportation (Low Guidance)
**Question 4:** Laura’s Garden (LG) has three citrus groves at locations **A, B, C** with available tones **A: 2750**, **B: 4000**, **C: 3000**. Processing plants at **D, E, F** have capacities **D: 2000**, **E: 6000**, **F: 2250**. Transportation cost is **€1.5 per ton per mile**. Distances (miles):

|   | D | E | F |
|---|---:|---:|---:|
| A | 21 | 50 | 40 |
| B | 35 | 30 | 22 |
| C | 55 | 20 | 25 |

Determine shipments from each grove to each plant to **minimize total transportation cost**.


<details>
<summary>💡 Hint</summary>

- **Decisions:** x[i,j] shipment from grove i to plant j.
- **Objective:** minimize 1.5 * sum(d[i,j] * x[i,j]).
- **Constraints:** supply equalities for A,B,C; capacity upper bounds for D,E,F; nonnegativity.

</details>

Decision: 9 decisions about from where to where to send products

mydictionary contains values and letters, but print(mydictionary.keys) prints all the possibilities between the letters
in these problems the structure, way of thinking, order of doing things is always THE SAME 
In Simulation problems its always a bit different depending on the problem
in the solution X is the value that we want to find. 
x=model.addVars(distances) tells gurobi to go to distance variables and make final decision variables so in the output we have \(X_ad, X_ae.... \)

In [ ]:
# 🧩 Your Turn (Q4)



## Q5 — Production & Inventory Planning (Low Guidance)
**Question 5:** Minimum Design manufactures lighting products and must plan production and inventory for the next **6 months**. Production costs vary by month; capacity also varies. Current inventory is **1800** units. Inventory holding cost is **€4** per unit-month. Inventory capacity is **≤ 6000** units at any time. Maintain at least **50%** of monthly production capacity each month, and keep at least **1500** units in inventory as safety stock. Data:

| Month | 1 | 2 | 3 | 4 | 5 | 6 |
|---|---:|---:|---:|---:|---:|---:|
| Unit production cost (€) | 250 | 253 | 255 | 253 | 250 | 255 |
| Demand | 1000 | 4500 | 6000 | 4500 | 3500 | 4000 |
| Maximum production | 4000 | 3500 | 4000 | 4500 | 4000 | 2500 |

Formulate and solve to minimize **production + holding** cost.


<details>
<summary>💡 Hint</summary>

- **Decisions:** production P[t], inventory I[t] for t=1..6.
- **Objective:** sum(c[t]*P[t] + 4*I[t]).
- **Constraints:** inventory balance, 1500 ≤ I[t] ≤ 6000, 0.5*Pmax[t] ≤ P[t] ≤ Pmax[t].
- Initial inventory I0 = 1800 (use it in the balance for month 1).

</details>

In [ ]:
# 🧩 Your Turn (Q5)
# Create sets, variables, and add balance and bounds.


## Q6 — Cash-Flow Investment Planning (Low Guidance)
**Question 6:** *As Greek As It Gets* (AGAIG) plans a new restaurant in Utrecht and needs a 6‑month **construction fund**. Costs: **€250,000 after 2 months**, **€250,000 at the end of 4 months**, **€300,000 at the end of 6 months**. Investment options:

| Investment | Available in Month | Months to Maturity | Yield at maturity |
|---|---:|---:|---:|
| A | 1,2,3,4,5,6 | 1 | 1.8% |
| B | 1,3,5 | 2 | 3.5% |
| C | 1,4 | 3 | 5.8% |
| D | 1 | 6 | 11% |

Decide investments to meet the payment schedule with the **minimum initial money**.


<details>
<summary>💡 Hint</summary>

- Let Z be the initial money (objective: minimize Z).
- Decision x[k,t] = amount invested in instrument k at month t (only when available).
- Carry cash S[t] month-to-month: S[t+1] = S[t] - sum_k x[k,t] + inflow[t] - payment[t].
- Inflow at month m is the sum of matured investments: x[k,t]*(1+yield_k) where t+mat_k = m.

</details>

In [ ]:
# 🧩 Your Turn (Q6)
# Implement the rolling-cash LP with variables Z, S[t], and x[k,t].
